In [3]:
import time 
import numpy as np
import pandas as pd
import random
import math

#comment out whole line of code Ctrl + /


def sim_taxi(lambda1, lambda2, aban1, aban2, time_coef, T):
    np.random.seed(0)
    
#     lambda1 = 1
#     lambda2 = 1.2
#     aban1 = 1
#     aban2 = 1
#     time_coef = 1
#     T = 100 #termination

    NumC_aban = 0
    total_utilization = 0 
    total_taxi = 0

    #event calendar vector of 3
    # explain each entry: 1st entry is taxi arrival, 2nd customer arrival and 3rd entry is termination time.
    EC = [0,0,0]


    #event types
    # 1st entry: is the event type, 2nd entry: is the ID of a taxi, 3rd entry: is the ID of customer
    # list grow as simulation changes. 
    ET = [[0,0,0],
         [0,0,0],
         [0,0,0]]

    #ET[0][0]

    LT = [] #list of taxi
    LC = []  #list of customers

                    #check len(LC) = NumC at arbitrary point 

    NumT = 0 #hold: the total number of taxi in the system at any particular state.
    total_T = 0 #hold: the total number of taxi in the system through out the simulation + the ID of the current taxi

    Num_trip = 0 # RECORD THE TOTAL NUMBER OF TRIP COMPLETED

    NumC = 0 #number of customers
    total_C = 0 #Customer ID
    
    

    EC[0] = np.random.exponential(1/lambda1) #arrival rate of taxi
    EC[1] = np.random.exponential(1/lambda2) #arrival rate of customers
    EC[2] = T #termination 

    ET[0][0] = 1
    ET[1][0] = 2
    ET[2][0] = 3 

    TNOW = 0 
    t_now = [] 
    uti_acc = []
    while TNOW < T:
        t_now.append(TNOW) 
        
        Tnext = min(EC)
        min_index = EC.index(Tnext)
        total_taxi += NumT*(Tnext - TNOW)
        num_of_busy_taxi = 0
        for i in range(len(LT)):
            num_of_busy_taxi += LT[i][0]
        total_utilization += num_of_busy_taxi*(Tnext-TNOW)

        print("Event Calendar = ",EC)
        print("Event type = ",ET)
        print("Min_index = ", min_index)

        print("type=", ET[min_index][0])
        print("TNOW = ",TNOW)
        print("Tnext = ",Tnext)
        print("List taxi = ",LT)
        print("NumT = ", NumT)
        print("List customer = ",LC)
        print("NumC = ", NumC)

        print("total_T = ", total_T) 
        print("total_C = ", total_C) 

        print("----------------------------------------------------")
        print("----------------------------------------------------")


        if ET[min_index][0] == 1: #event type: Taxi arrival
            TNOW = Tnext
            X = np.random.uniform(0,2) #add arrival locations
            Y = np.random.uniform(0,2)
            NumT += 1 #update number of taxi
            total_T += 1
            LT.append([0, X, Y, total_T]) #update list taxi

            EC[0] = TNOW + np.random.exponential(1/lambda1) #update EC
            ET.append([4, total_T, 0]) #taxi finishing shift/abandonment of taxi
            EC.append(TNOW + np.random.exponential(1/aban1))
            Assigned_1 = 0
            Min_Distance = 10000
            Customer_assigned_1 = 0

            #change all customer_assigned to the ID of the customer instead total_C

            #loop to find which taxi is the closest when NumC is not zero.
            if NumC > 0: 
                for i in range(NumC): #range take value from 0 -> NumC-1
                    if LC[i][0] == 0:
                        Assigned_1 = 1
                        Distance = math.sqrt((LC[i][1]-X)**2 + (LC[i][2]-Y)**2)
                        if Distance < Min_Distance:
                            Min_Distance = Distance 
                            Customer_assigned_1 = i
#                             print("Customer_assigned_1", Customer_assigned_1)
#                     print("Min_Distance = ",Min_Distance)

            # assignd that closest taxi to a customer and update LT(change from iddle = 0 to busy =1), EC, LC and ET.
            if Assigned_1 == 1: # this mean that if Assigned == True
                #LT[Customer_assigned][0] = 1 #make ID which is total_T become the index
                num_of_busy_taxi += 1
                LT[NumT-1][0] = 1 # reassigning the 0-th index of this matrix to the value of NumT line 52.
                EC.append(TNOW + time_coef*(Min_Distance + math.sqrt((LC[Customer_assigned_1][1]-LC[Customer_assigned_1][3])**2 +
                                                                     (LC[Customer_assigned_1][2]-LC[Customer_assigned_1][4])**2)))

                LC[Customer_assigned_1][0] = 1
                ET.append([7, total_T, LC[Customer_assigned_1][5]])

                FR1 = -1 #found row 
                for i in range(len(EC)):
                    if ET[i][2] == LC[Customer_assigned_1][5]: #this condition check if this event is an abandonment and it also check if the event is correspond to the customer we are deal with.
                        # Customer assigned + 1 is because if Customer assigned = 0 then we know that it is the first customer that is going to abandon.
                        FR1 = i # want to find which row of the EC and the ET correspond the abandonment of the customer 
                        break

                    #Delete/ remove and item from a list of EC and ET
                EC.pop(FR1)
                ET.pop(FR1) 

            #check this line above 

        #assigning customers to taxi    
        elif ET[min_index][0] == 2: #event type: Customer arrival
            TNOW = Tnext
            X1 = np.random.uniform(0,2)
            Y1 = np.random.uniform(0,2)
            Des_X = np.random.uniform(0,2) #Generate destination points when customer arrive
            Des_Y = np.random.uniform(0,2)

            NumC += 1
            total_C += 1
            LC.append([0, X1, Y1, Des_X, Des_Y, total_C])
            EC[1] = TNOW + np.random.exponential(1/lambda2)


            Min_Distance = 10000
            found_2 = 0
            Taxi_found_2 = 0

            #Check if is there an idle taxi ?     
            if NumT > 0:
                for i in range(NumT): #loop through the list of taxi to find the closest taxi and assign it to the customer.

                    if LT[i][0] == 0: #if taxi i-th is idle, then we found one but still need to check for min distance. 
                        found_2 = 1
                        Distance = math.sqrt((LT[i][1]-X1)**2 + (LT[i][2]-Y1)**2)
                        if Distance < Min_Distance:
                            Min_Distance = Distance 
                            Taxi_found_2 = i

                            #dont use break here because if we break then we would never find the closest taxi, but rather we found a taxi.

            if found_2 == 1:
                #LC[Taxi_found][0] = 1
                LC[NumC-1][0] = 1 #customer i-th become busy 
    #                 print("Taxi_found = ", Taxi_found)
                num_of_busy_taxi += 1
                EC.append(TNOW + time_coef*(Min_Distance + math.sqrt((LC[NumC-1][1]-LC[NumC-1][3])**2 + 
                                                                        (LC[NumC-1][2]-LC[NumC-1][4])**2)))
                ET.append([7, LT[Taxi_found_2][3], total_C]) #keep track of the ID of the taxi.
                LT[Taxi_found_2][0] = 1
                #ET.append([7, Taxi_found, NumC]) =  ET.append([7, NumT, Customer_assigned])

            else: #if found == 0 then assign departure time for customers.
                ET.append([5, 0, total_C])
                EC.append(TNOW + np.random.exponential(1/aban2))


            #We need to assign the 0-th taxi to customer.
            #The first bracket denote the index for each taxi.

                #Think about assigning a taxi that is going to abandon but has to finish service 

        elif ET[min_index][0] == 3: #event type: Termination 

            # out put performance measure
            TNOW = T
            Trip_complete = Num_trip #record the total number of trip completed by all taxi.

            Taxi_throughput = total_T/np.random.exponential(1/lambda1)
            utilization = total_utilization/total_taxi 
            
            print("Total taxi =", total_T)
            print("Total customer =", total_C)
            print("TOTAL BUSY TAXI =", num_of_busy_taxi)
            
            print("total_taxi = ", total_taxi)
            
            
            print("Total number of trip completion =", Trip_complete)
            print("rate of abandonment = ", NumC_aban/T)
            print("Total customer who abandon = ",NumC_aban)
            print("utilization=",utilization)
            
            
            
            print("t_now = ", t_now)
            print("uti_acc = ", uti_acc)
            print(len(t_now))
            print(len(uti_acc))
            

            print("END SIMULATION.")
            print("---------------------------------------------------------")
            # #total_trip_complete = 
            # #num_cus_abandon =
            # #num_taxi_abandon = 
            # #average_customer_cycle_time = number_customer/total_cus_left
            # #average_taxi_cycle_time = number_taxi/total_taxi_left 
            return total_T, total_C, num_of_busy_taxi, Trip_complete, NumC_aban/T, utilization,\
        t_now, uti_acc
            
       


        elif ET[min_index][0] == 4: #event type: taxi finish shift and leave
            TNOW = Tnext
            t_found_4 = 0 #identify the taxi that is going to leave
            taxi_considered_4 = ET[min_index][1]

            print("HERE I AM IN ET == 4, TRYING TO DELETE OR ELSE THIS TAXI:",ET[min_index])
            print("ET = ",ET)
            print("ET[min_index] = ", ET[min_index])
            print("taxi_considered = ",taxi_considered_4)
            print(LT)
            print("EC_min_index = ",EC[min_index])

            for k in range(len(LT)):
    #                 print("k=",k, "LT[k][3]=", LT[k][3] , "taxi_considered =",taxi_considered_4)
                if LT[k][3] == taxi_considered_4: #this check the id: id(LT[k][3]) == id(taxi_considered)
                    t_found_4 = k #index of the taxi ID.
                    break
    #             print("taxi status = ", LT[t_found_4][0])

            if LT[t_found_4][0] == 1: #if the taxi we are considering is busy
    #                 print("taxi busy = ", taxi_considered_4)

                for i in range(len(EC)):
    #                     print("i=",i) 
    #                     print("lenET = ",len(ET))
                    if ET[i][0] == 7 and ET[i][1] == taxi_considered_4:
                        if EC[min_index] == EC[i]: #if EC of taxi abandoment = to EC of taxi complete the final trip.
    #                         print("index is", i, EC[min_index], EC[i])
                            Num_trip += 1 #record the total trip completions.
                            print("A TRIP HAS COMPLETED.")
                            print("ET[i][2] = ",ET[i][2])
                            print("LC=",LC)

                            for j in range(len(LC)):
                                if ET[i][2] == LC[j][5]: #check if the ID of the considered customer is actually the customer considered
                                    LC.pop(j)
                                    NumC -= 1

            #                                   print("deleted", i)
                                    break
                            EC.pop(i) #delete i-th index of the trip completed that is associated with EC and ET.
                            ET.pop(i)

                            EC.pop(min_index) #delete index of the taxi abandonemnt event that is associated with EC and ET.
                            ET.pop(min_index)
                            LT.pop(t_found_4)
                            NumT -= 1
                            num_of_busy_taxi -= 1


                        else:
                            EC[min_index] = EC[i]
        #               print("postion 2")
                        break


            else: #if there is no customer to arrive before taxi finish shift then just delete taxi
                EC.pop(min_index)
                ET.pop(min_index)
                LT.pop(t_found_4)
                NumT -= 1
                num_of_busy_taxi -= 1
    #       print("Im here")


        elif ET[min_index][0] == 5: #event type: customer run out of patience 
            #delete appropriate column from LC, ET, EC, update numC.
            TNOW = Tnext
            NumC_aban += 1
            Customer_found_5 = 0
            Customer_depart_5 = ET[min_index][2]
            for i in range(NumC):
                if LC[i][5] == Customer_depart_5:
                    Customer_found_5 = i
                    break
                    EC.pop(i)
                    ET.pop(i)

            LC.pop(Customer_found_5) # NumC - 1 should be after LC.pop and the same thing apply for LC
            NumC -= 1
            EC.pop(min_index)
            ET.pop(min_index)


            
            
            
        elif ET[min_index][0] == 7: #event type: taxi taking customer to destination
            TNOW = Tnext 
            Customer_found_7 = 0
            Num_trip += 1 #this variable hold the total number of trip completion.


            #Find the customer who is going to finsh his trip
            Customer_complete_7 = ET[min_index][2] #denote the ID of the customer who is going to finsh his trip. 
            for i in range(NumC):
                if LC[i][5] == Customer_complete_7:
                    Customer_found_7 = i
                    break
            print("Customer_found_7=", Customer_found_7)
            #Find the driver that is taking this customer to finish the trip
            Taxi_complete_7 = ET[min_index][1] #denote the ID of the taxi who is taking the customer to finish her service.
            Taxi_found_7 = 0
            
            for j in range(NumT):
                if LT[j][3] == Taxi_complete_7:
                    Taxi_found_7 = j
                    break
            print("Taxi_found_7=", Taxi_found_7)
            print("TAxi: complete, found", Taxi_complete_7, Taxi_found_7)
            print("ET[min_index][2]",ET[min_index][2])
            print("HERE I am at ET 7 inside the very first FOR loop")
            print("Customer_found", Customer_found_7)
            print("I HAVE JUST FINISHED SERVING THIS CUSTOMER: ", Customer_complete_7)

            X_cur = LC[Customer_found_7][3] #update thecurrent positition of taxi i-th after dropping off customer j.
            Y_cur = LC[Customer_found_7][4]
            
            #Update the status and the location of the current taxi to the dropped off coordinates
            print(ET[min_index])
            print("Taxi_found_7", Taxi_found_7)
            print("LT", LT)
            LT[Taxi_found_7][0] = 0
            LT[Taxi_found_7][1] = X_cur
            LT[Taxi_found_7][2] = Y_cur

    #             print("X_cur = ", X_cur)
    #             print("Y_cur = ", Y_cur)

            #assign a different customer to same taxi
            Min_Distance = 10000
            assigned_new_7 = 0
            Assigned_7 = 0 # this indicate whether or not we have assgined a customer. This is as same as Assigned= 0.

            print("IM going to delete: ", Customer_found_7)
            LC.pop(Customer_found_7)
            EC.pop(min_index)
            
            #if we don't record ET[min_index] to the variable taxi_index then after we pop min_index from ET, the algorithm  
            # will assign a wrong taxi to an idle customer. 
            
            taxi_index = ET[min_index][1]
            ET.pop(min_index)
            print("I just deleted: ", Customer_found_7)
            NumC -= 1
            print("Customers after deleting:", LC)
            
            # the problem is we try to assign customer to driver, try assign driver to customer instead. This don't work because
            # the problem is not in how we are assigning them, but rather which taxi is being assigned to a customer. 
            # and we don't want to assign a busy taxi to a idle customer.
            
            if NumC > 0: #check to see if there is any other customer close by.
                 for k in range(NumC): #range take value from 0 -> NumC-1
                    if LC[k][0] == 0:
                        Assigned_7 = 1
                        #check if the location of the i-th taxi that just dropped off customer 1 is close to customer 2
                        Distance = math.sqrt((LC[k][1]-X_cur)**2 + (LC[k][2]-Y_cur)**2)
                        if Distance < Min_Distance:
                            Min_Distance = Distance 
                            assigned_new_7 = k
                            print("In computing distance; this is the favourite:", LC[assigned_new_7])

            if Assigned_7 == 1:
                num_of_busy_taxi += 1
                LT[Taxi_found_7][0] = 1
                LC[assigned_new_7][0] = 1 # make customer i-th become busy
    #                 print("assigned_new=",assigned_new_7)
                EC.append(TNOW + time_coef*(Min_Distance + math.sqrt((LC[assigned_new_7][1]-LC[assigned_new_7][3])**2 +
                                                                        (LC[assigned_new_7][2]-LC[assigned_new_7][4])**2)))
                ET.append([7, taxi_index, LC[assigned_new_7][5]])

                #The below step make sure that we delete the abandonment of customer once they have been assigned a taxi.
                FR7 = -1 #found row
                for m in range(len(EC)):
                    print("im in FR7")
                    print("m =",m)
                    print("assigned_new_7 = ",assigned_new_7)
                    
                    if ET[m][2] == LC[assigned_new_7][5]: #this condition check if this event is an abandonment and it also check if the event is correspond to the customer we are deal with.
                        #Customer assigned + 1 is because if Customer assigned = 0 then we know that it is the first customer that is going to abandon.
                        FR7 = m # want to find which row of the EC and the ET correspond the abandonment of the customer
                        break

                #Delete/ remove and item from a list of EC and ET
                EC.pop(FR7)
                ET.pop(FR7)
                print("I have just executed if Assigned = 1 ")
#                 print("FR =", FR)
#                 print("m = ", m)

            else:
                print("Taxi_found_7", Taxi_found_7)
                LT[Taxi_found_7][0] = 0 # min_index never used on LT and LC
                LT[Taxi_found_7][1] = X_cur
                LT[Taxi_found_7][2] = Y_cur
                num_of_busy_taxi -= 1
                print(" I COULD not find a customer to match and I have just executed else condition of ET==7")

#             ET.pop(min_index)
#             EC.pop(min_index)



        
        print("Event Calendar = ",EC)
        print("Event type = ",ET)
        print("Min_index = ", min_index)

        print("TNOW = ",TNOW)
        print("Tnext = ",Tnext)
        print("List taxi = ",LT)
        print("NumT = ", NumT)
        print("List customer = ",LC)
        print("NumC = ", NumC)

        print("total_T = ", total_T) 
        print("total_C = ", total_C) 
        print(Num_trip)

        print("----------------------------------------------------")
        print("----------------------------------------------------")
        
        
        uti_acc.append(weird_division(total_utilization,total_taxi))
        


    #Number of customer should be equal to the total number of of Event type 5 and 7.
    #Number of Taxi should be equal to the total number of of Event type 4.
    #Don't schedule a new event when you delete: dont do ET or EC.append
    # NumC -1 or NumT - 1 always after pop.LC and pop.LT
    
def weird_division(n, d): # for when dividing by zero due to low rate of arrivals.
    return n / d if d else 0

In [4]:
def drange(start, stop, step):
    r = start
    while r < stop:
        yield r
        r += step

# for index in drange(1,10,1):
#     sim_taxi(lambda1 = 1, lambda2 = index, aban1=1, aban2=1, time_coef=1, T=100)
#     print(index)

# acc_total_T = []
# acc_total_C = []
# acc_num_of_busy_taxi = []
# acc_Trip_complete = []
# acc_abandonment = []
# acc_utilization = []
# for index in drange(1,4,1):
#     total_T, total_C, num_of_busy_taxi, Trip_complete, abandonment, utilization = sim_taxi(lambda1 = 1, lambda2 = index, aban1=1, aban2=1, time_coef=1, T=100)
#     acc_total_T.append(total_T)
#     acc_total_C.append(total_C)
#     acc_num_of_busy_taxi.append(num_of_busy_taxi)
#     acc_Trip_complete.append(Trip_complete)
#     acc_abandonment.append(abandonment)
#     acc_utilization.append(utilization)
    
#     print(index)

# print("Printing accumulated elements")
# print(acc_total_T)
# print(acc_total_C)
# print(acc_utilization)

sim_taxi(lambda1 = 1, lambda2 = 1, aban1 =1, aban2 = 1, time_coef=1, T=100)

# for index in np.arange(0.01, 10, 0.5):
#     sim_taxi(lambda1 = 1, lambda2 = index, aban1=1, aban2=1, time_coef=1, T=100)
#     print(index)



Event Calendar =  [0.79587450816311, 1.2559307629658378, 100]
Event type =  [[1, 0, 0], [2, 0, 0], [3, 0, 0]]
Min_index =  0
type= 1
TNOW =  0
Tnext =  0.79587450816311
List taxi =  []
NumT =  0
List customer =  []
NumC =  0
total_T =  0
total_C =  0
----------------------------------------------------
----------------------------------------------------
Event Calendar =  [1.3469229992586094, 1.2559307629658378, 100, 1.8340338031067194]
Event type =  [[1, 0, 0], [2, 0, 0], [3, 0, 0], [4, 1, 0]]
Min_index =  0
TNOW =  0.79587450816311
Tnext =  0.79587450816311
List taxi =  [[0, 1.2055267521432877, 1.0897663659937937, 1]]
NumT =  1
List customer =  []
NumC =  0
total_T =  1
total_C =  0
0
----------------------------------------------------
----------------------------------------------------
Event Calendar =  [1.3469229992586094, 1.2559307629658378, 100, 1.8340338031067194]
Event type =  [[1, 0, 0], [2, 0, 0], [3, 0, 0], [4, 1, 0]]
Min_index =  1
type= 2
TNOW =  0.79587450816311
Tnext =

(102,
 81,
 0,
 62,
 0.19,
 0.7021574446862198,
 [0,
  0.79587450816311,
  1.2559307629658378,
  1.3469229992586094,
  1.4206083541714571,
  1.8340338031067194,
  2.824826902935006,
  3.487434229993533,
  3.94517715318405,
  4.071051565014593,
  4.8651411838805565,
  5.17242153697101,
  5.400706902181312,
  5.461713130064202,
  5.907231622795441,
  6.362087831376172,
  6.5982945367101795,
  6.606852088714578,
  7.058954145060964,
  7.166589257596255,
  7.293255660693922,
  7.451279869365488,
  7.453119787570432,
  7.458669792503525,
  7.472685264670552,
  7.574830151335116,
  7.575625204544677,
  7.615601507028187,
  7.74199014874729,
  7.746766591889992,
  7.794530838670447,
  7.908035449841299,
  7.998302464825809,
  8.526741982302667,
  8.578087693977919,
  8.921179675877632,
  9.019829989557882,
  9.02931648173281,
  9.16116070357711,
  9.779075317863072,
  10.120675910818044,
  10.122589960698166,
  10.323026347598098,
  10.327732882168796,
  10.613753859696237,
  11.0038049571076

In [62]:
LC = [[1, 0.5425041535237283, 0.807585481334669, 0.7996242800186615, 1.3427669573403063, 60], [1, 1.278373798450785, 0.7983222905095462, 0.8635202553086385, 1.2290553996206415, 61], [1, 0.6420859800864338, 0.05990064980949872, 1.4745084851929546, 0.2195689161250014, 63]]
LT = [[1, 1.2695726458673895, 1.91828450395595, 84], [1, 0.05838046969782651, 1.069833709854168, 85], [1, 0.7301997541200196, 0.3811338298801361, 86]]


X = np.random.uniform(0,2) #add arrival locations
Y = np.random.uniform(0,2)


Assigned_1 = 0
Min_Distance = 10000
assigned = 0


for i in range(len(LC)): #range take value from 0 -> NumC-1
    if LC[i][0] == 0:
        Assigned_1 = 1
        Distance = math.sqrt((LC[i][1]-X)**2 + (LC[i][2]-Y)**2)
        if Distance < Min_Distance:
            Min_Distance = Distance
            
            assigned = i          
            print("i am " ,assigned)
             